In [1]:
## install required packages
!pip install swig
!pip install wrds
!pip install pyportfolioopt
## install finrl library
!pip install -q condacolab
!apt-get update -y -qq && apt-get install -y -qq cmake libopenmpi-dev python3-dev zlib1g-dev libgl1-mesa-glx swig
!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

^C


'apt-get'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [8]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

%matplotlib inline
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from stable_baselines3.common.logger import configure
from finrl.meta.data_processor import DataProcessor

from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from pprint import pprint

import sys
sys.path.append("../FinRL")

import itertools

### Create Folders

from finrl import config
from finrl import config_tickers
import os
from finrl.main import check_and_make_directories
from finrl.config import (
    DATA_SAVE_DIR,
    TRAINED_MODEL_DIR,
    TENSORBOARD_LOG_DIR,
    RESULTS_DIR,
    INDICATORS,
    TRAIN_START_DATE,
    TRAIN_END_DATE,
    TEST_START_DATE,
    TEST_END_DATE,
    TRADE_START_DATE,
    TRADE_END_DATE,
)
check_and_make_directories([DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR])



# from config.py, TRAIN_START_DATE is a string
TRAIN_START_DATE
# from config.py, TRAIN_END_DATE is a string
TRAIN_END_DATE

TRAIN_START_DATE = '2010-01-04'
TRAIN_END_DATE = '2021-10-01'
TRADE_START_DATE = '2021-10-01'
TRADE_END_DATE = '2023-05-05'

tickers = ['XLB','XLE','XLF','XLI','XLK','XLP','XLU','XLV','XLY']

df = YahooDownloader(start_date = TRAIN_START_DATE,
                     end_date = TRADE_END_DATE,
                     ticker_list = tickers).fetch_data()

df.sort_values(['date','tic'],ignore_index=True).head()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (30213, 8)


,date,open,high,low,close,volume,tic,day
0,2010-01-04,33.580002,34.020000,33.450001,25.488928,7567500,XLB,0
1,2010-01-04,57.919998,58.810001,57.790001,38.080162,16928400,XLE,0
2,2010-01-04,11.795288,11.965881,11.770918,9.385947,92942347,XLF,0
3,2010-01-04,28.090000,28.320000,27.959999,21.880188,7471500,XLI,0
4,2010-01-04,23.139999,23.290001,23.100000,19.150221,8449400,XLK,0


In [11]:
#split data ticker wise
df_XLB = df[df['tic']=='XLB'][['date','tic','close']]
df_XLE = df[df['tic']=='XLE'][['date','tic','close']]
df_XLF = df[df['tic']=='XLF'][['date','tic','close']]
df_XLI = df[df['tic']=='XLI'][['date','tic','close']]
df_XLK = df[df['tic']=='XLK'][['date','tic','close']]
df_XLP = df[df['tic']=='XLP'][['date','tic','close']]
df_XLU = df[df['tic']=='XLU'][['date','tic','close']]
df_XLV = df[df['tic']=='XLV'][['date','tic','close']]
df_XLY = df[df['tic']=='XLY'][['date','tic','close']]

In [14]:
# Dlinear
D = pd.DataFrame({'XLB':df_XLB['close'].values,
                    'XLE':df_XLE['close'].values,
                    'XLF':df_XLF['close'].values,
                    'XLI':df_XLI['close'].values,
                    'XLK':df_XLK['close'].values,
                    'XLP':df_XLP['close'].values,
                    'XLU':df_XLU['close'].values,
                    'XLV':df_XLV['close'].values,
                    'XLY':df_XLY['close'].values},index=df_XLB['date'].values)

D

,XLB,XLE,XLF,XLI,XLK,XLP,XLU,XLV,XLY
2010-01-04,25.488928,38.080162,9.385947,21.880188,19.150221,18.718464,19.481915,25.213638,25.227169
2010-01-05,25.571354,38.390972,9.558456,21.957457,19.125528,18.725483,19.249992,24.966366,25.319664
2010-01-06,26.005892,38.850708,9.577625,22.003801,18.911566,18.711449,19.362823,25.221615,25.353292
2010-01-07,25.803612,38.792439,9.782079,22.243311,18.837500,18.711449,19.275064,25.309351,25.563526
2010-01-08,26.163239,39.044956,9.724579,22.598709,18.960951,18.648277,19.256264,25.349237,25.555117
...,...,...,...,...,...,...,...,...,...
2023-04-28,80.550003,85.129997,33.169998,100.000000,150.830002,77.440002,68.980003,133.529999,147.860001
2023-05-01,80.510002,84.169998,33.080002,100.540001,150.919998,77.500000,69.099998,134.220001,146.399994
2023-05-02,79.730003,80.510002,32.330002,99.489998,149.509995,77.290001,68.260002,133.639999,146.550003
2023-05-03,78.849998,78.989998,31.959999,99.110001,148.250000,76.660004,68.070000,133.460007,145.389999


In [15]:
D.to_csv('D.csv')

In [101]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from tqdm import tqdm

# Define the LSTM model
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size)
        self.linear = nn.Linear(hidden_size, output_size)

    def forward(self, input_seq):
        lstm_out, _ = self.lstm(input_seq.reshape(len(input_seq), 1, -1))
        predictions = self.linear(lstm_out.reshape(len(input_seq), -1))
        return predictions[-1]


# Convert data to DataLoader
train_data = train_dataset.copy()
train_data = train_data.drop('date', axis=1)
train_data = train_data.values
train_data = torch.FloatTensor(train_data)
train_loader = DataLoader(train_data, batch_size=100, shuffle=True)

# Move model and data to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LSTM(9, 512, 9).to(device)
loss_function = nn.MSELoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)


# Train LSTM
def train_LSTM(model, loss_function, optimizer, epochs, train_loader):
    model.train()
    losses = []
    for i in tqdm(range(epochs)):
        for seq in train_loader:
            optimizer.zero_grad()
            model.hidden_cell = (torch.zeros(1, 1, model.hidden_size).to(device),
                                 torch.zeros(1, 1, model.hidden_size).to(device))
            y_pred = model(seq[:-1])
            loss = loss_function(y_pred, seq[-1])
            loss.backward()
            optimizer.step()
        losses.append(loss.item())
    return model

# Train LSTM
model = train_LSTM(model, loss_function, optimizer, 100, train_loader)


100%|██████████| 100/100 [02:41<00:00,  1.62s/it]


In [105]:
train_data = train_dataset.copy()
train_data = train_data.drop('date', axis=1)
train_data = train_data.values
train_data = torch.FloatTensor(train_data)

test_data = test_dataset.copy()
test_data = test_data.drop('date', axis=1)
test_data = test_data.values
test_data = torch.FloatTensor(test_data)

def predict_LSTM(model, seq):
    model.eval()
    model.hidden_cell = (torch.zeros(1, 1, model.hidden_size).to(device),
                         torch.zeros(1, 1, model.hidden_size).to(device))
    with torch.no_grad():
        seq = seq.unsqueeze(0).to(device)  # Add extra dimension for batch size
        predictions = model(seq).squeeze().tolist()  # Get predictions for all columns
    return predictions

train_predictions = [predict_LSTM(model, seq) for seq in train_data]
test_predictions = [predict_LSTM(model, seq) for seq in test_data]

# Reshape train_predictions array
train_predictions = np.array(train_predictions).reshape(-1, len(tickers))

# Convert predictions to a DataFrame
train_predictions = pd.DataFrame(train_predictions, columns=tickers)
train_predictions


,XLB,XLE,XLF,XLI,XLK,XLP,XLU,XLV,XLY
0,25.761951,29.025887,11.009329,30.768970,31.204149,24.130501,22.763273,36.874676,45.401600
1,25.747593,29.012495,11.006999,30.754784,31.190145,24.121658,22.752878,36.854816,45.377563
2,25.651062,28.892529,10.963563,30.637272,31.066925,24.026770,22.666616,36.711296,45.202553
3,25.675840,28.916668,10.969390,30.667603,31.095924,24.046476,22.688055,36.748173,45.247570
4,25.598166,28.823154,10.936817,30.574274,30.998854,23.972868,22.618116,36.632835,45.106033
...,...,...,...,...,...,...,...,...,...
3458,27.923491,31.138285,11.504090,32.929642,33.261833,25.851643,24.227732,39.789360,48.764523
3459,27.974251,31.205244,11.527669,33.002831,33.334877,25.914869,24.292231,39.864552,48.875751
3460,27.988342,31.221506,11.533232,33.020443,33.348110,25.929216,24.307056,39.880745,48.897816
3461,27.969786,31.198301,11.528335,32.996677,33.322170,25.907806,24.287857,39.854229,48.856358


In [106]:
test_predictions = np.array(test_predictions).reshape(-1, len(tickers))
test_predictions = pd.DataFrame(test_predictions, columns=tickers)
test_predictions

,XLB,XLE,XLF,XLI,XLK,XLP,XLU,XLV,XLY
0,27.980049,31.215588,11.533685,33.013824,33.345181,25.922667,24.302393,39.874989,48.885494
1,28.037086,31.291533,11.559493,33.097214,33.426426,25.992990,24.375368,39.961998,49.010971
2,28.029821,31.283087,11.556625,33.086899,33.419651,25.984806,24.366329,39.952606,48.997356
3,28.019558,31.268394,11.550990,33.071259,33.401955,25.971144,24.353926,39.936222,48.971703
4,28.005589,31.250057,11.542399,33.050411,33.383175,25.954090,24.333744,39.915825,48.944805
...,...,...,...,...,...,...,...,...,...
395,28.584459,32.066582,11.896908,33.911121,34.310936,26.591290,25.075172,40.804939,50.210686
396,28.581383,32.058323,11.899674,33.906635,34.304207,26.585825,25.070848,40.796257,50.201370
397,28.552652,32.004269,11.878631,33.853245,34.240589,26.550638,25.026613,40.736301,50.113663
398,28.541384,31.984762,11.874403,33.836132,34.219070,26.538031,25.012316,40.714966,50.085121


In [107]:
#concat train and test predictions
predictions = pd.concat([train_predictions, test_predictions], axis=0).reset_index(drop=True)


In [108]:
predictions

,XLB,XLE,XLF,XLI,XLK,XLP,XLU,XLV,XLY
0,25.761951,29.025887,11.009329,30.768970,31.204149,24.130501,22.763273,36.874676,45.401600
1,25.747593,29.012495,11.006999,30.754784,31.190145,24.121658,22.752878,36.854816,45.377563
2,25.651062,28.892529,10.963563,30.637272,31.066925,24.026770,22.666616,36.711296,45.202553
3,25.675840,28.916668,10.969390,30.667603,31.095924,24.046476,22.688055,36.748173,45.247570
4,25.598166,28.823154,10.936817,30.574274,30.998854,23.972868,22.618116,36.632835,45.106033
...,...,...,...,...,...,...,...,...,...
3858,28.584459,32.066582,11.896908,33.911121,34.310936,26.591290,25.075172,40.804939,50.210686
3859,28.581383,32.058323,11.899674,33.906635,34.304207,26.585825,25.070848,40.796257,50.201370
3860,28.552652,32.004269,11.878631,33.853245,34.240589,26.550638,25.026613,40.736301,50.113663
3861,28.541384,31.984762,11.874403,33.836132,34.219070,26.538031,25.012316,40.714966,50.085121


In [133]:
import pandas as pd

# Shift the dates by one day
shifted_dates = Prediction_df['date'].unique().tolist()
shifted_dates.append('2023-05-05')

predictions['date'] = shifted_dates


In [134]:
predictions = predictions[['date'] + tickers]

#predictions data = date > TRAIN_START_DATE
predictions_data = predictions[predictions['date'] >= TRAIN_START_DATE]

predictions_data

,date,XLB,XLE,XLF,XLI,XLK,XLP,XLU,XLV,XLY
505,2010-01-04,24.939920,28.023102,10.648551,29.730457,30.129486,23.320890,21.978334,35.615200,43.857552
506,2010-01-05,24.939800,28.026789,10.650089,29.730600,30.131260,23.322559,21.978504,35.615677,43.858547
507,2010-01-06,24.959515,28.051937,10.660620,29.756908,30.157597,23.343828,21.997366,35.644882,43.895947
508,2010-01-07,24.982729,28.079615,10.668865,29.783108,30.185556,23.364338,22.016697,35.678833,43.937828
509,2010-01-08,25.006002,28.107727,10.680913,29.812414,30.215260,23.388519,22.038296,35.713261,43.980732
...,...,...,...,...,...,...,...,...,...,...
3858,2023-05-01,28.584459,32.066582,11.896908,33.911121,34.310936,26.591290,25.075172,40.804939,50.210686
3859,2023-05-02,28.581383,32.058323,11.899674,33.906635,34.304207,26.585825,25.070848,40.796257,50.201370
3860,2023-05-03,28.552652,32.004269,11.878631,33.853245,34.240589,26.550638,25.026613,40.736301,50.113663
3861,2023-05-04,28.541384,31.984762,11.874403,33.836132,34.219070,26.538031,25.012316,40.714966,50.085121


In [135]:
predictions_data.to_csv('LSTM_Prediction.csv',index=False)

In [136]:
predictions_data = pd.read_csv('LSTM_Prediction.csv')

In [137]:
#transform the dataframe to date, tic, close format
predictions_data = pd.melt(predictions_data, id_vars=['date'], value_vars=tickers)
predictions_data = predictions_data.rename(columns={'variable': 'tic', 'value': 'close'})

#transform the dataframe ordered by date
predictions_data = predictions_data.sort_values(['date','tic'],ignore_index=True)



In [139]:
predictions_data.to_csv('LSTM_Prediction.csv',index=False)

In [140]:
predictions_data

,date,tic,close
0,2010-01-04,XLB,24.939920
1,2010-01-04,XLE,28.023102
2,2010-01-04,XLF,10.648551
3,2010-01-04,XLI,29.730457
4,2010-01-04,XLK,30.129486
...,...,...,...
30217,2023-05-05,XLK,34.211105
30218,2023-05-05,XLP,26.531570
30219,2023-05-05,XLU,25.008202
30220,2023-05-05,XLV,40.709332
